# Creating a Medical triage workflow with TigerGraph
<ol>


# Set Up


In [2]:
!python --version
!pip install pyTigerGraph

Python 2.7.16


# Exploring pyTigerGraph

In [4]:
!pip install pyTigerGraph

In [5]:
import pyTigerGraph as tg # Importing

# Create a Connection
conn = tg.TigerGraphConnection(host="https://healthnet.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

In [3]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))

gaierror: [Errno 8] nodename nor servname provided, or not known

In [37]:
# Create Edges (published and references) and Vertices (source, article and event)
print(conn.gsql('''
  CREATE VERTEX condition(PRIMARY_ID id STRING, name STRING, desciption STRING, icd_code STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX symptom(PRIMARY_ID id STRING, name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX precaution(PRIMARY_ID id STRING, description STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
  CREATE VERTEX specialist(PRIMARY_ID name STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"

  CREATE DIRECTED EDGE has_symptom(FROM condition, TO symptom) WITH REVERSE_EDGE="reverse_has_symptom"
  CREATE DIRECTED EDGE has_precaution(FROM condition, TO precaution) WITH REVERSE_EDGE="reverse_has_precaution"
  CREATE DIRECTED EDGE refer_to(FROM condition, TO specialist) WITH REVERSE_EDGE="reverse_refer_to"

  ''', options=[]))

print(conn.gsql('''CREATE GRAPH healthnet(condition, symptom, precaution, specialist, has_symptom, has_precaution, refer_to)''', options=[])) # Create the Graph



The graph newsgraph is created.


In [38]:
# Edit the Connect Credentials for Loading Data
conn.graphname = "healthnet"
conn.apiToken = conn.getToken(conn.createSecret())

Now we can import the CSV media rating into the graph.

What is a ICD-10 diagnosis code?
Used for medical claim reporting in all healthcare settings, ICD-10-CM is a standardized classification system of diagnosis codes that represent conditions and diseases, related health problems, abnormal findings, signs and symptoms, injuries, external causes of injuries and diseases, and social circumstances